In [68]:
# import libaray
import mne
import numpy as np
#import matplotlib as mp
import pandas as pd
from mne import Epochs, pick_types, events_from_annotations
from mne.channels import read_layout
from mne.preprocessing import ICA
from mne.io import concatenate_raws
import matplotlib.pyplot as plt
from matplotlib import mlab

import os
import glob

In [78]:
# block 1
#read egg data  of one subject one day   ,5 pre stim feedbacks and 5 post stim feed baks

frame=pd.DataFrame()
dataarray=np.zeros([10,14,15000])# i changed this value from 15000
df=pd.DataFrame()
basepath='S01/day2'
iter=0
for trail in os.listdir(os.path.join(basepath)):
    for file in  os.listdir(os.path.join(basepath,trail)):
        if(trail[:-2]=='neurofeedback' and (int(trail[-2:])<=5 or int(trail[-2:])>=11)):
            df=pd.read_csv(os.path.join(basepath,trail,file),sep='\t',header=None)
            df=df.T
            print(basepath,trail,file)
            print(df.shape)
            data=df.values;
            dataarray[iter,:,:]=data[:,0:15000] #  i changed this value 15000
            iter+=1
dataarray.shape

S01/day2 neurofeedback01 anodal_S01_day2_nf01_EEGraw_and_condition.txt
(14, 20176)
S01/day2 neurofeedback02 anodal_S01_day2_nf02_EEGraw_and_condition.txt
(14, 18032)
S01/day2 neurofeedback03 anodal_S01_day2_nf03_EEGraw_and_condition.txt
(14, 19152)
S01/day2 neurofeedback04 anodal_S01_day2_nf04_EEGraw_and_condition.txt
(14, 20320)
S01/day2 neurofeedback05 anodal_S01_day2_nf05_EEGraw_and_condition.txt
(14, 20320)
S01/day2 neurofeedback11 anodal_S01_day2_nf11_EEGraw_and_condition.txt
(14, 20000)
S01/day2 neurofeedback12 anodal_S01_day2_nf12_EEGraw_and_condition.txt
(14, 18928)
S01/day2 neurofeedback13 anodal_S01_day2_nf13_EEGraw_and_condition.txt
(14, 20320)
S01/day2 neurofeedback14 anodal_S01_day2_nf14_EEGraw_and_condition.txt
(14, 18992)
S01/day2 neurofeedback15 anodal_S01_day2_nf15_EEGraw_and_condition.txt
(14, 19520)


(10, 14, 15000)

In [79]:
# block 2
#eegdata=np.load('./anodal/S03day1.npy')
eegdata=dataarray
cols=['Fcz', 'Fc2', 'Fc4', 'Fc6', 'Cz', 'C2', 'C4', 'C6', 'Cpz', 'Cp2', 'Cp4', 'Cp6','Cz','stim']
eegdata[:,:13,:]=eegdata[:,:13,:]/1000000
#df=pd.DataFrame(data=data.T,,columns=cols)
#print(eegdata[1,2,:20])
#datastack=np.reshape(data,(10*14,18000))

In [80]:
# block 3
# read one of pre stim  feedbaks and one post stime feedback
#filter the  feedbacks
#plot raw data
#plot psd topographis maps

#oneepoch variable contains  prestim  all MI epochs (shape =[nevents,nchan,nsamples])
#twoepcoh variable contains  prestim  all MI epochs (shape =[nevents,nchan,nsamples])




sampling_freq = 128  # in Hertz
Fs=sampling_freq
[ntrail,nchann,nsamp]=eegdata.shape
%matplotlib 
sampling_freq = 128  # in Hertz
ch_names = ['FCz', 'FC2', 'FC4', 'FC6', 'Cz', 'C2', 'C4', 'C6', 'CPz', 'CP2', 'CP4', 'CP6','A1','STI 001']
ch_types = ['eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg','eeg','stim']

info = mne.create_info(ch_names=ch_names, sfreq=sampling_freq, ch_types=ch_types);
#for trail in range(0,ntrail):
raw= mne.io.RawArray(eegdata[0,:,:], info)  # reading neurofeddback 2 which correspond to prestim 3 n3
raw.set_montage("standard_1020")
#raw.set_eeg_reference(['Cz'])
raw.filter(l_freq=8, h_freq=22) #filter the fraw data from 8 to 22 hertz
#raw.plot_psd(fmax=50);  #plot psd of
start, stop = raw.time_as_index([100, 115])  # 100 s to 115 s data segment
data, times = raw[:, start:stop]

#plt.plot(raw.get_data()[0])




#data, times = raw[2:20:3, start:stop]  # access underlying data
#plotraw.get_data()[0]
raw.plot(); #plot raw data pre
reject= dict(  eeg=150e-6  )     # 150 µV
                     
event_id = {'ready': 1, 'MI': 2, 'postrest': 3}

events = mne.find_events(raw, 'STI 001');
#fig = mne.viz.plot_events(events, event_id=id, sfreq=raw.info['sfreq'],first_samp=raw.first_samp)

raw.info['bads'] += [ 'A1']
picks = mne.pick_types(raw.info, eeg=True, stim=False,exclude='bads')
epochs = mne.Epochs(raw, events, event_id, tmin=-0.2, tmax=0.5,  picks=picks, reject=reject)
oneepoch=epochs['MI'].get_data()
#print(epochs)

#epochs['MI'].plot_psd(fmin=2, fmax=30)
#epochs.plot_psd_topomap(ch_type='eeg', normalize=True);
epochs['MI'].plot_psd_topomap(ch_type='eeg', normalize=True);


evoked = epochs['MI'].average()
#print(evoked)
#evoked.plot(time_unit='s')





#post MI data
info = mne.create_info(ch_names=ch_names, sfreq=sampling_freq, ch_types=ch_types);
#for trail in range(0,ntrail):
raw= mne.io.RawArray(eegdata[6,:,:], info)  # reading neurofeddback 8 which correspon to poststim nf 14 n14
raw.set_montage("standard_1020")
#raw.set_eeg_reference(['Cz'])
raw.filter(l_freq=8, h_freq=22)
#raw.plot_psd(fmax=50);
start, stop = raw.time_as_index([100, 115])  # 100 s to 115 s data segment
data, times = raw[:, start:stop]

plt.plot(raw.get_data()[0])




#data, times = raw[2:20:3, start:stop]  # access underlying data
#plotraw.get_data()[0]
raw.plot();  # print raw data post
reject= dict(  eeg=150e-6  )     # 150 µV
                     
event_id = {'ready': 1, 'MI': 2, 'postrest': 3}

events = mne.find_events(raw, 'STI 001');
##fig = mne.viz.plot_events(events, event_id=event_id, sfreq=raw.info['sfreq'],first_samp=raw.first_samp)

raw.info['bads'] += [ 'A1']
picks = mne.pick_types(raw.info, eeg=True, stim=False,exclude='bads')
epochs = mne.Epochs(raw, events, event_id, tmin=-0.2, tmax=0.5,  picks=picks, reject=reject)
epochs['MI'].plot_psd_topomap(ch_type='eeg', normalize=True);
twoepoch=epochs['MI'].get_data()

#TIME FREQUENCEY ANAYLSIS
# n_cycles = 2  # number of cycles in Morlet wavelet
# freqs = np.arange(7, 30, 3)  # frequencies of interest
# from mne.time_frequency import tfr_morlet  # noqa
# power, itc = tfr_morlet(epochs['MI'], freqs=freqs, n_cycles=n_cycles,return_itc=True, decim=3, n_jobs=1)
# power.plot([power.ch_names.index('FC2')]); #TF graph for FC2
print(oneepoch.shape)
print(twoepoch.shape)
print(min([oneepoch.shape[0],twoepoch.shape[0]]))

Using matplotlib backend: Qt5Agg
Creating RawArray with float64 data, n_channels=14, n_times=15000
    Range : 0 ... 14999 =      0.000 ...   117.180 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 22 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 22.00 Hz
- Upper transition bandwidth: 5.50 Hz (-6 dB cutoff frequency: 24.75 Hz)
- Filter length: 213 samples (1.664 sec)

23 events found
Event IDs: [1 2 3]
Not setting metadata
Not setting metadata
23 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 8 events and 91 original time points ...
0 bad epochs dropped
Loading data for 8 event

In [81]:
# block 4
#preM1re and postMI shape=[nchan,nsample,nevent] ; reshapeing  data for further SCP and PSD
#epoch=epochs.get_data()
#print(epochs[0:3])
minev=min([oneepoch.shape[0],twoepoch.shape[0]])

[ev,ch,sp]=oneepoch.shape;
preMI=np.zeros([ch,sp,ev])
#print(ev,ch,sp)
collectev=np.zeros([ev,sp])
for channel in range(0,ch):
    for evt in range(0,ev):
        collectev[evt,:]=oneepoch[evt,channel,:]
    preMI[channel,:,:]=collectev[:,:].T
    
#print(preMI.shape)

preMIre=preMI[:,:,0:minev]
#print(preMIre.shape)

[ev,ch,sp]=twoepoch.shape;
postMI=np.zeros([ch,sp,ev])
#print(ev,ch,sp)
collectev=np.zeros([ev,sp])
for channel in range(0,ch):
    for evt in range(0,ev):
        collectev[evt,:]=twoepoch[evt,channel,:]
    postMI[channel,:,:]=collectev[:,:].T
postMIre=postMI[:,:,0:minev]    
print(postMIre.shape)
print(preMIre.shape)


# # for post MI
# raw= mne.io.RawArray(data[5,:,:], info)
# raw.filter(l_freq=12, h_freq=22)
# events = mne.find_events(raw, 'STI 001');
# raw.set_montage("standard_1020")
# raw.set_eeg_reference(['A1'])
# print(events.shape); 
# print(events)
# picks = mne.pick_channels(raw.info["ch_names"], ['FCz', 'FC2', 'FC4', 'FC6', 'Cz', 'C2', 'C4', 'C6']);
# tmin, tmax = -3, 4  # define epochs around events (in s)
# event_ids = dict(MI=2)  # map event IDs to tasks
# epochs = mne.Epochs(raw, events, event_ids, tmin , tmax ,
#                     picks=picks,baseline=None, preload=True);
# epochs[0].plot_psd()
# epochs.plot_psd_topomap()
#     #print(epochs[0:3])
# epoch=epochs.get_data()

# #print(epochs[0:3])

# oneepoch=epochs.get_data()
# [ev,ch,sp]=oneepoch.shape;
# postMI=np.zeros([ch,sp,ev])
# print(ev,ch,sp)
# collectev=np.zeros([ev,sp])
# for channel in range(0,ch):
#     for evt in range(0,ev):
#         collectev[evt,:]=oneepoch[evt,channel,:]
#     postMI[channel,:,:]=collectev[:,:].T
    
# print(postMI.shape)


#raw = concatenate_raws([mne.io.RawArray(data[f,:,:], info) for f in range(0,ntrail)]);
    #raw.plot(n_channels=3, scalings='auto', title='Auto-scaled Data from arrays',
       #  show=True, block=True)
# lets plot filtered data
#note trail 8 data currpted exlude it.
# datanp=np.delete(datanp,[7],0)
# print(datanp.shape)
# events = mne.find_events(raw, 'STI 001');
#print(events);
# picks = mne.pick_channels(raw.info["ch_names"], [ 'Cz', 'C2', 'C4', 'C6','STI 001']);
# tmin, tmax = 0, 4  # define epochs around events (in s)
# event_ids = dict(MI=2)  # map event IDs to tasks

# epochs = mne.Epochs(raw, events, event_ids, tmin , tmax ,
#                     picks=picks,baseline=None, preload=True);
# #print(epochs[0:3])
# oneepoch=epochs.get_data()
# print(oneepoch[:,4,:])
# print(oneepoch.shape)
#datanp=np.delete(datanp,6,0)
#print(datanp.shape)
# [ntrail,nchan,nsample] = datanp.shape
# #print(datanp[trail,0:12,:15])
# for trail in range(0,ntrail,1):
#     plt.figure(trail)
#     for channel in range(0,nchan,1):
#         plt.plot(datanp[trail,channel,0:10*sampling_freq])
    


(12, 91, 8)
(12, 91, 8)


In [82]:
# block 5

def psd(trials):
    '''
    Calculates for each trial the Power Spectral Density (PSD).
    
    Parameters
    ----------
    trials : 3d-array (channels x samples x trials)
        The EEG signal
    
    Returns
    -------
    trial_PSD : 3d-array (channels x PSD x trials)
        the PSD for each trial.  
    freqs : list of floats
        Yhe frequencies for which the PSD was computed (useful for plotting later)
    '''
    
    [nchannels,nsamples,ntrials] = trials.shape
  
    trials_PSD = np.zeros((nchannels, 46, ntrials))

    # Iterate over trials and channels
    for trial in range(ntrials):
        for ch in range(nchannels):
            # Calculate the PSD
            (PSD, freqs) = mlab.psd(trials[ch,:,trial], NFFT=int(nsamples), Fs=Fs)
            trials_PSD[ch, :, trial] = PSD.ravel()
                
    return trials_PSD, freqs

In [83]:
# block 6
cl_lab=['pre', 'post']
cl1 = cl_lab[0]
cl2 = cl_lab[1]
trials={}
trials[cl1]=preMIre
trials[cl2]=postMIre



psd_r, freqs = psd(trials[cl1])
psd_f, freqs = psd(trials[cl2])
print(psd_f.shape)
print(freqs.shape)
trials_PSD = {cl1: psd_f}
#print(trials_PSD.keys())
print(nchann)
plt.figure('pre')
for chan in range(0,nchann-2):
    plt.plot(np.mean(psd_r[chan,:,:] ,axis=1))
plt.figure('post')
for chan in range(0,nchann-2):
    plt.plot(np.mean(psd_f[chan,:,:] ,axis=1))

(12, 46, 8)
(46,)
14


In [84]:
# block 7
def plot_psd(trials_PSD, freqs, chan_ind, chan_lab=None, maxy=None):
    '''
    Plots PSD data calculated with psd().
    
    Parameters
    ----------
    trials : 3d-array
        The PSD data, as returned by psd()
    freqs : list of floats
        The frequencies for which the PSD is defined, as returned by psd() 
    chan_ind : list of integers
        The indices of the channels to plot
    chan_lab : list of strings
        (optional) List of names for each channel
    maxy : float
        (optional) Limit the y-axis to this value
    '''
    plt.figure(figsize=(12,5))
    
    nchans = len(chan_ind)
    
    # Maximum of 3 plots per row
    nrows = np.ceil(nchans / 3)
    ncols = min(3, nchans)
    
    # Enumerate over the channels
    for i,ch in enumerate(chan_ind):
        # Figure out which subplot to draw to
        plt.subplot(nrows,ncols,i+1)
    
        # Plot the PSD for each class
        for cl in trials.keys():
            plt.plot(freqs, np.mean(trials_PSD[cl][ch,:,:], axis=1), label=cl,linewidth=5)
    
        # All plot decoration below...
        
        plt.xlim(1,30)
        
        if maxy != None:
            plt.ylim(0,maxy)
    
        plt.grid()
    
        plt.xlabel('Frequency (Hz)')
        
        if chan_lab == None:
            plt.title('Channel %d' % (ch+1))
        else:
            plt.title(chan_lab[i])

        plt.legend()
        
    plt.tight_layout()

In [85]:
# psd_r, freqs = psd(trials[cl1])
# psd_f, freqs = psd(trials[cl2])
# trials_PSD = {cl1: psd_r, cl2: psd_f}

# plot_psd(
#     trials_PSD,
#     freqs,
#     [0,5,-1],
#     chan_lab=['left', 'center', 'right'],
#     maxy=300
# )


In [86]:
# block 8
from numpy import linalg

def cov(trials):
    ''' Calculate the covariance for each trial and return their average '''
    [nchannels, nsamples, ntrials] = preMIre.shape
    covs = [ trials[:,:,i].dot(trials[:,:,i].T) / nsamples for i in range(ntrials) ]
    return np.mean(covs, axis=0)

def whitening(sigma):
    ''' Calculate a whitening matrix for covariance matrix sigma. '''
    U, l, _ = linalg.svd(sigma)
    return U.dot( np.diag(l ** -0.5) )

def csp(trials_r, trials_f):
    '''
    Calculate the CSP transformation matrix W.
    arguments:
        trials_r - Array (channels x samples x trials) containing right hand movement trials
        trials_f - Array (channels x samples x trials) containing foot movement trials
    returns:
        Mixing matrix W
    '''
    cov_r = cov(trials_r)
    cov_f = cov(trials_f)
    P = whitening(cov_r + cov_f)
    B, _, _ = linalg.svd( P.T.dot(cov_f).dot(P) )
    W = P.dot(B)
    return W

def apply_mix(W, trials):
    ''' Apply a mixing matrix to each trial (basically multiply W with the EEG signal matrix)'''
    [nchannels, nsamples, ntrials] = preMIre.shape
    trials_csp = np.zeros((nchannels, nsamples, ntrials))
    for i in range(ntrials):
        trials_csp[:,:,i] = W.T.dot(trials[:,:,i])
    return trials_csp

In [87]:
# block 9
W = csp(preMIre, postMI)
trials_csp = {cl1: apply_mix(W, trials[cl1]),
              cl2: apply_mix(W, trials[cl2])}

In [88]:
# block 10
psd_r, freqs = psd(trials_csp[cl1])
psd_f, freqs = psd(trials_csp[cl2])
trials_PSD = {cl1: psd_r, cl2: psd_f}

plot_psd(trials_PSD, freqs, [0,11,-1], chan_lab=['CSP of MI', 'middle component', 'last component'], maxy=0.75 )